In [3]:
import os, pathlib, time

files = pathlib.Path(r"D:\ig_pipeline").rglob('cad/*/*/processed.max')
files_with_mtime = [(os.path.getmtime(f), f) for f in files]
files_with_mtime.sort()
for mtime, f in files_with_mtime:
    formatted_mtime = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(mtime))
    print(f"{formatted_mtime}: {f}")

2024-10-14 13:44:42: D:\ig_pipeline\cad\scenes\Wainscott_0_garden\processed.max
2024-10-14 13:44:45: D:\ig_pipeline\cad\scenes\Wainscott_0_int\processed.max
2024-10-14 13:44:47: D:\ig_pipeline\cad\scenes\Wainscott_1_int\processed.max
2024-10-14 13:45:50: D:\ig_pipeline\cad\objects\batch-00\processed.max
2024-10-14 13:45:58: D:\ig_pipeline\cad\objects\batch-01\processed.max
2024-10-14 13:46:06: D:\ig_pipeline\cad\objects\batch-02\processed.max
2024-10-14 13:46:21: D:\ig_pipeline\cad\objects\batch-03\processed.max
2024-10-14 13:46:28: D:\ig_pipeline\cad\objects\batch-04\processed.max
2024-10-14 13:46:35: D:\ig_pipeline\cad\objects\batch-05\processed.max
2024-10-14 13:46:42: D:\ig_pipeline\cad\objects\batch-06\processed.max
2024-10-14 13:46:50: D:\ig_pipeline\cad\objects\batch-07\processed.max
2024-10-14 13:47:21: D:\ig_pipeline\cad\objects\batch-08\processed.max
2024-10-14 13:47:28: D:\ig_pipeline\cad\objects\batch-09\processed.max
2024-10-14 13:47:35: D:\ig_pipeline\cad\objects\batch-10